In [2]:
import pandas as pd
import numpy as np

val = pd.read_csv("../../../Insight-Wave/StockPrediction/data/Fundamental+Technical Data/STOCK_DATA_WEEKLY/ADVANC.csv")
predict = pd.read_csv("../../../Insight-Wave/StockPrediction/model/ADVANC/Fundamental+LDA News+LDA Twitter+GDELT V1/XGBModel_lags2_lags_past_covariates2_output_chunk_length1.csv")
val_df = val[['Close']].iloc[-35-35-1:-35]
predict_df = predict[['Closing_Price']]

In [10]:
def calculate_directional_accuracy(actual, forecast):  
    acc = 0
    for i in range(len(forecast)):
        actual_change = actual[i + 1] - actual[i]
        predicted_change = forecast[i] - actual[i]
        if (actual_change > 0 and predicted_change > 0) or (actual_change < 0 and predicted_change < 0) or (actual_change == 0 and predicted_change == 0):
            acc += 1

    da = round(acc / len(forecast) * 100, 2)
    return da

def calculate_directional_accuracy_with_thresholds(actual, forecast):
    thresholds = [0, 5, 10] 
    combinations = [(up, -down) for up in thresholds for down in thresholds if not (up > 0 and down > 0)]
    results = []
    for up, down in combinations:
        acc = 0
        for i in range(len(forecast)):
            actual_change = ((actual[i + 1] - actual[i]) / actual[i]) * 100
            predicted_change = ((forecast[i] - actual[i]) / actual[i]) * 100
            if (actual_change >= up and predicted_change >= up) or \
                (actual_change <= down and predicted_change <= down):
                acc += 1

        da = round(acc / len(forecast) * 100, 2)
        results.append((up, abs(down), da))
    
    return results

def cal_err_and_acc(predicted, val, condition=True):

    val.reset_index(drop=True, inplace=True)
    predicted.reset_index(drop=True, inplace=True)

    actual = val['Close']
    forecast = predicted['Closing_Price']
    
    dir_acc = calculate_directional_accuracy_with_thresholds(actual, forecast) #calculate_directional_accuracy(actual, forecast)
    print(type(dir_acc))
    
    if condition:
        return dir_acc
    #else:
        # print(f"MAPE = {mape_error:.2f} %")
        # print(f"RMSE = {rmse_error:.2f} %\n")
        # print(f"Directional Accuracy = {dir_acc:.2f} %")

try:
    avg_dir = cal_err_and_acc(predict_df, val_df, True)
except Exception as e:
    print("Error when calculate error ", e)
avg_dir

<class 'list'>


[(0, 0, 68.57), (0, 5, 31.43), (0, 10, 31.43), (5, 0, 37.14), (10, 0, 37.14)]

In [14]:
type(avg_dir[0][2])

float